# **Instagram Clone User Data Analysis**

\*\*Use the ig\_clone database\*\*

In [3]:
USE ig_clone
GO

Commands completed successfully.

Total execution time: 00:00:00.014

## **Rewarding the Oldest Customers**

Let's reward the Instagram users who have been around the longest. For that, we wish to find the 10 oldest customers and reward them.

In [5]:
SELECT TOP(10) WITH TIES * FROM users 
ORDER BY created_at ASC;

(10 rows affected)

Total execution time: 00:00:00.032

id,username,created_at
80,Darby_Herzog,2016-05-06 00:14:21.190
67,Emilio_Bernier52,2016-05-06 13:04:29.960
63,Elenor88,2016-05-08 01:30:40.677
95,Nicole71,2016-05-09 17:30:22.370
38,Jordyn.Jacobson2,2016-05-14 07:56:25.837
71,Nia_Haag,2016-05-14 15:38:50.230
40,Rafael.Hickle2,2016-05-19 09:51:25.780
58,Aurelie71,2016-05-31 06:20:56.910
88,Clint27,2016-06-02 21:40:09.557
91,Bethany20,2016-06-03 23:31:53.323


The 10 users who have stayed around the longest on Instagram had opened their accounts between 6th May and 3rd June 2016.

## **Scheduling an Ad Campaign**

We wish to schedule an ad campaign on a particular day. This should be on the day when most users register for an Instagram account.

In [13]:
SELECT DATENAME(dw, created_at) as 'day of the week', COUNT(*) as 'total users registered'
FROM users 
GROUP BY DATENAME(dw, created_at)
ORDER BY 'total users registered' DESC;

(7 rows affected)

Total execution time: 00:00:00.018

day of the week,total users registered
Sunday,16
Thursday,16
Friday,15
Monday,14
Tuesday,14
Wednesday,13
Saturday,12


We can see that almost all days of the week are equally popular for registration, among which the most popular are Sunday and Thursday. We can target Sunday for the campaign as it is also a weekend and users are most likely to spend more time on Instagram.

## **Targeting Users for Email Campaign**

It is assumed that there are many users who have either never posted a photo or posted very few photos on Instagram. In order to increase their loyalty, we should aim an email campaign at users who posted 2 or less photos to encourage them to use Instagram and share more of their photos with the world.

In [22]:
SELECT username, COALESCE(COUNT(photos.id), 0) as photos_posted 
FROM users 
LEFT JOIN photos 
ON users.id = photos.user_id
GROUP BY username
HAVING COALESCE(COUNT(photos.id), 0) < 3
ORDER BY photos_posted;

Warning: Null value is eliminated by an aggregate or other SET operation.

(57 rows affected)

Total execution time: 00:00:00.010

username,photos_posted
Aniya_Hackett,0
Bartholome.Bernhard,0
Bethany20,0
Darby_Herzog,0
David.Osinski47,0
Duane60,0
Esmeralda.Mraz57,0
Esther.Zulauf61,0
Franco_Keebler64,0
Hulda.Macejkovic,0


We can see there are more than 50 users who have posted 2 or less photos on Instagram, among which 26 did not even post a single photo and so, we will make them our target audience for the email campaign.

## **Popular Photo Contest**

We have decided to run a new contest to see who can get the most likes on a single photo on Instagram till now. We will be distributing gifts to the top 3 users with most likes on a singe photo of theirs.

In [34]:
SELECT TOP(3) WITH TIES username, photos.id AS photo_id, photos.image_url, COUNT(*) AS total_likes
FROM photos
JOIN likes
ON likes.photo_id = photos.id
JOIN users
ON photos.user_id = users.id
GROUP BY username, photos.id, photos.image_url
ORDER BY total_likes DESC;

(3 rows affected)

Total execution time: 00:00:00.029

username,photo_id,image_url,total_likes
Zack_Kemmer93,145,https://jarret.name,48
Malinda_Streich,127,https://celestine.name,43
Adelle96,182,https://dorcas.biz,43


The users above are the winners of the contest as they have the 3 most liked photos on Instagram till now.

## **Average User Activity**

The investors of Instagram wants to know how many times does the average user posts a photo. It is to be found by 'total number of photos/total number of users'.

In [41]:
SELECT ROUND((SELECT 1.0 * COUNT(*) FROM photos)/(SELECT 1.0 * COUNT(*) FROM users), 2, 2) as avg_user_activity;

(1 row affected)

Total execution time: 00:00:00.013

avg_user_activity
2.570000000000000


It can be seen that a user posts 2.57 photos on average.

## **Meeting Posts Target**

The investors have set a target for the total number of posts per month at 100 for the continuation of their investment. Hence, we should keep a check if we are able to meet the target.

In [50]:
SELECT DATEPART(YEAR, created_at) AS year, DATENAME(MONTH, created_at) AS month, COUNT(*) AS total_posts
FROM photos 
GROUP BY DATEPART(YEAR, created_at), DATENAME(MONTH, created_at);

(1 row affected)

Total execution time: 00:00:00.014

year,month,total_posts
2022,November,257


Since we have only posts of November 2022, we can see that we have exceeded the target set by our investors by a healthy margin of 157 posts till now.

## **User Ranking**

We want to rank the users based on the number of photos they have posted so that we can send the most loyal users links to various promotions as appreciation of their loyalty.

In [46]:
SELECT username, COUNT(*) AS photos_posted, DENSE_RANK() OVER(ORDER BY COUNT(*) DESC) as rank
FROM users 
JOIN photos 
ON users.id = photos.user_id
GROUP BY username
ORDER BY rank;

(74 rows affected)

Total execution time: 00:00:00.020

username,photos_posted,rank
Eveline95,12,1
Clint27,11,2
Cesar93,10,3
Delfina_VonRueden68,9,4
Aurelie71,8,5
Jaime53,8,5
Donald.Fritsch,6,6
Florence99,5,7
Harrison.Beatty50,5,7
Colten.Harris76,5,7


We can see that the top 3 ranked users have posted 10 or more photos and these will be the ones who will sent promotional links for their brand loyalty.